In [18]:
# Install OS dependencies.  This only needs to be run once for each new notebook instance.
!pip install PyAthena

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
!pip install pydotplus

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
from pyathena import connect
from pyathena.util import as_pandas
from __future__ import print_function

# Import libraries
import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import *
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas import DataFrame
from sklearn import neighbors
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import boto3
from io import StringIO
from io import BytesIO
from botocore.client import ClientError
# below is used to print out pretty pandas dataframes
from IPython.display import display, HTML
%matplotlib inline


s3 = boto3.resource('s3')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name
athena_query_results_bucket = 'aws-athena-query-results-'+account_id+'-'+region

try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
except ClientError:
    bucket = s3.create_bucket(Bucket=athena_query_results_bucket)
    print('Creating bucket '+athena_query_results_bucket)
cursor = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp').cursor()


# The Glue database name of your MIMIC-III parquet data
gluedatabase="mimiciii"

# location of the queries to generate aline specific materialized views
aline_path = './'

# location of the queries to generate materialized views from the MIMIC code repository
concepts_path = './SQL/'

In [21]:
query = 'select * from DATABASE.mp_data_6hr'
cursor.execute(query.replace("DATABASE", gluedatabase))
data_6hr_df = as_pandas(cursor)

query = 'select * from DATABASE.mp_data_12hr'
cursor.execute(query.replace("DATABASE", gluedatabase))
data_12hr_df = as_pandas(cursor)

query = 'select * from DATABASE.mp_data_18hr'
cursor.execute(query.replace("DATABASE", gluedatabase))
data_18hr_df = as_pandas(cursor)

In [22]:
columns_6hr=list(data_6hr_df)
columns_12hr=list(data_12hr_df)
columns_18hr=list(data_18hr_df)

In [23]:
reduced_6hr_df=data_6hr_df.dropna(axis=1,how='all',thresh=18073)
reduced_12hr_df=data_12hr_df.dropna(axis=1,how='all',thresh=18073)
reduced_18hr_df=data_18hr_df.dropna(axis=1,how='all',thresh=18073)

In [24]:
columns_6hr=list(reduced_6hr_df)
columns_12hr=list(reduced_12hr_df)
columns_18hr=list(reduced_18hr_df)

In [25]:
misscount_6hr= reduced_6hr_df.isnull().sum(axis=1)
misscount_12hr= reduced_12hr_df.isnull().sum(axis=1)
misscount_18hr= reduced_18hr_df.isnull().sum(axis=1)

In [26]:
misscount_6hr.describe()
misscount_12hr.describe()
misscount_18hr.describe()

count    49632.000000
mean        12.745084
std         12.037876
min          0.000000
25%          3.000000
50%         11.000000
75%         19.000000
max         98.000000
dtype: float64

In [27]:
reduced_featurs_6hr= reduced_6hr_df.drop(['icustay_id'], axis=1)
reduced_featurs_12hr= reduced_12hr_df.drop(['icustay_id'], axis=1)
reduced_featurs_18hr= reduced_18hr_df.drop(['icustay_id'], axis=1)

In [28]:
missvalue_6hr= reduced_featurs_6hr.isnull().sum(axis='columns')
missvalue_12hr= reduced_featurs_12hr.isnull().sum(axis='columns')
missvalue_18hr= reduced_featurs_18hr.isnull().sum(axis='columns')
missvalue_6hr.shape
missvalue_12hr.shape
missvalue_18hr.shape
print(missvalue_6hr.describe())
print(missvalue_12hr.describe())
print(missvalue_18hr.describe())

count    49632.000000
mean        18.526676
std         17.688804
min          0.000000
25%          4.000000
50%         13.000000
75%         30.000000
max         92.000000
dtype: float64
count    49632.000000
mean        15.544971
std         14.477661
min          0.000000
25%          4.000000
50%         16.000000
75%         19.000000
max         98.000000
dtype: float64
count    49632.000000
mean        12.745084
std         12.037876
min          0.000000
25%          3.000000
50%         11.000000
75%         19.000000
max         98.000000
dtype: float64


In [29]:
print('reduced features 6hr')
print(reduced_featurs_6hr.shape)
reduced_featurs_6hr = reduced_featurs_6hr.dropna(thresh=71)
print(reduced_featurs_6hr.shape)
print('reduced features 6hr')

print('reduced features 12hr')
print(reduced_featurs_12hr.shape)
reduced_featurs_12hr = reduced_featurs_12hr.dropna(thresh=71)
print(reduced_featurs_12hr.shape)
print('reduced features 12hr')

print('reduced features 18hr')
print(reduced_featurs_18hr.shape)
reduced_featurs_18hr = reduced_featurs_18hr.dropna(thresh=71)
print(reduced_featurs_18hr.shape)
print('reduced features 18hr')

reduced features 6hr
(49632, 102)
(38739, 102)
reduced features 6hr
reduced features 12hr
(49632, 108)
(45800, 108)
reduced features 12hr
reduced features 18hr
(49632, 108)
(47899, 108)
reduced features 18hr


In [30]:
cat_6hr = reduced_featurs_6hr[['gender', 'endotrachflag_mean', 'hospital_expire_flag' ]].copy().reset_index(drop=True)
reduced_featurs_6hr.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag' ], axis=1, inplace=True)
reduced_featurs_6hr = reduced_featurs_6hr.reset_index(drop=True)

cat_12hr = reduced_featurs_12hr[['gender', 'endotrachflag_mean', 'hospital_expire_flag' ]].copy().reset_index(drop=True)
reduced_featurs_12hr.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag' ], axis=1, inplace=True)
reduced_featurs_12hr = reduced_featurs_12hr.reset_index(drop=True)


cat_18hr = reduced_featurs_18hr[['gender', 'endotrachflag_mean', 'hospital_expire_flag' ]].copy().reset_index(drop=True)
reduced_featurs_18hr.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag' ], axis=1, inplace=True)
reduced_featurs_18hr = reduced_featurs_18hr.reset_index(drop=True)

In [31]:
reduced_featurs_6hr.fillna(reduced_featurs_6hr.mean(), inplace=True)
cat_6hr.fillna(0, inplace=True)

reduced_featurs_12hr.fillna(reduced_featurs_12hr.mean(), inplace=True)
cat_12hr.fillna(0, inplace=True)

reduced_featurs_18hr.fillna(reduced_featurs_18hr.mean(), inplace=True)
cat_18hr.fillna(0, inplace=True)

In [32]:
reduced_featurs_6hr = pd.concat([reduced_featurs_6hr, cat_6hr], axis=1).reset_index()
reduced_featurs_6hr.to_csv("./Data/data_6hr_df.csv")

reduced_featurs_12hr = pd.concat([reduced_featurs_12hr, cat_12hr], axis=1).reset_index()
reduced_featurs_12hr.to_csv("./Data/data_12hr_df.csv")

reduced_featurs_18hr = pd.concat([reduced_featurs_18hr, cat_18hr], axis=1).reset_index()
reduced_featurs_18hr.to_csv("./Data/data_18hr_df.csv")

In [33]:
print(reduced_featurs_6hr.shape)
print(reduced_featurs_12hr.shape)
print(reduced_featurs_18hr.shape)

(38739, 103)
(45800, 109)
(47899, 109)


In [34]:
reduced_featurs_6hr.to_csv("./Data/ICU_6hr_df.csv")
reduced_featurs_12hr.to_csv("./Data/ICU_12hr_df.csv")
reduced_featurs_18hr.to_csv("./Data/ICU_18hr_df.csv")

In [35]:
Deceased_6hr = len(reduced_featurs_6hr[reduced_featurs_6hr['hospital_expire_flag'] == 1])
print("\nDeceased 6hr:   %d" %Deceased_6hr)
print("\nDishcharged 6hr:   %d" %len(reduced_featurs_6hr[reduced_featurs_6hr['hospital_expire_flag'] == 0]))

Deceased_12hr = len(reduced_featurs_12hr[reduced_featurs_12hr['hospital_expire_flag'] == 1])
print("\nDeceased 12hr:   %d" %Deceased_12hr)
print("\nDishcharged 12hr:   %d" %len(reduced_featurs_12hr[reduced_featurs_12hr['hospital_expire_flag'] == 0]))

Deceased_18hr = len(reduced_featurs_18hr[reduced_featurs_18hr['hospital_expire_flag'] == 1])
print("\nDeceased 18hr:   %d" %Deceased_18hr)
print("\nDishcharged 18hr:   %d" %len(reduced_featurs_18hr[reduced_featurs_18hr['hospital_expire_flag'] == 0]))


Deceased 6hr:   4783

Dishcharged 6hr:   33956

Deceased 12hr:   5342

Dishcharged 12hr:   40458

Deceased 18hr:   5466

Dishcharged 18hr:   42433


In [36]:
Discharged_indices_6hr = reduced_featurs_6hr[reduced_featurs_6hr['hospital_expire_flag'] == 0].index
random_indices_6hr = np.random.choice(Discharged_indices_6hr,Deceased_6hr, replace=False)
Deceased_indices_6hr = reduced_featurs_6hr[reduced_featurs_6hr.hospital_expire_flag == 1].index
under_sample_indices_6hr = np.concatenate([Deceased_indices_6hr,random_indices_6hr])
under_sample_6hr = reduced_featurs_6hr.loc[under_sample_indices_6hr]


Discharged_indices_12hr = reduced_featurs_12hr[reduced_featurs_12hr['hospital_expire_flag'] == 0].index
random_indices_12hr = np.random.choice(Discharged_indices_12hr,Deceased_12hr, replace=False)
Deceased_indices_12hr = reduced_featurs_12hr[reduced_featurs_12hr.hospital_expire_flag == 1].index
under_sample_indices_12hr = np.concatenate([Deceased_indices_12hr,random_indices_12hr])
under_sample_12hr = reduced_featurs_12hr.loc[under_sample_indices_12hr]


Discharged_indices_18hr = reduced_featurs_18hr[reduced_featurs_18hr['hospital_expire_flag'] == 0].index
random_indices_18hr = np.random.choice(Discharged_indices_18hr,Deceased_18hr, replace=False)
Deceased_indices_18hr = reduced_featurs_18hr[reduced_featurs_18hr.hospital_expire_flag == 1].index
under_sample_indices_18hr = np.concatenate([Deceased_indices_18hr,random_indices_18hr])
under_sample_18hr = reduced_featurs_18hr.loc[under_sample_indices_18hr]

In [37]:
under_sample_12hr.head()

,index,subject_id,hadm_id,dbsource,expire_flag,deathtime_hours,age,ethnicity,admission_type,icustay_num,...,potassium_max,ptt_max,inr_max,sodium_max,bun_max,wbc_max,urineoutput,gender,endotrachflag_mean,hospital_expire_flag
3,3,9,150750,carevue,1,107.0,41,UNKNOWN/NOT SPECIFIED,EMERGENCY,1,...,2.9,41.822596,1.600081,138.0,27.384997,13.010696,1670.0,M,1.0,1
5,5,12,112213,carevue,1,283.0,72,WHITE,ELECTIVE,1,...,4.6,47.500000,1.600000,145.0,28.000000,8.400000,655.0,M,1.0,1
12,12,21,111970,carevue,1,194.0,87,WHITE,EMERGENCY,2,...,3.4,43.500000,1.800000,140.0,50.000000,41.200000,252.0,M,0.0,1
18,18,31,128652,carevue,1,157.0,72,WHITE,EMERGENCY,1,...,3.7,28.500000,1.100000,133.0,11.000000,13.010696,3345.0,M,1.0,1
40,40,61,189535,carevue,1,320.0,55,WHITE,EMERGENCY,2,...,3.3,41.822596,1.600081,143.0,16.000000,0.100000,2735.0,M,0.0,1


In [38]:
print("\nUnder sampled Deceased 6hr:   %d" %len(under_sample_6hr[under_sample_6hr['hospital_expire_flag'] == 1]))
print("\nUnder sampled Dishcharged 6hr:   %d" %len(under_sample_6hr[under_sample_6hr['hospital_expire_flag'] == 0]))

print("\nUnder sampled Deceased 12hr:   %d" %len(under_sample_12hr[under_sample_12hr['hospital_expire_flag'] == 1]))
print("\nUnder sampled Dishcharged 12hr:   %d" %len(under_sample_12hr[under_sample_12hr['hospital_expire_flag'] == 0]))

print("\nUnder sampled Deceased 18hr:   %d" %len(under_sample_18hr[under_sample_18hr['hospital_expire_flag'] == 1]))
print("\nUnder sampled Dishcharged 18hr:   %d" %len(under_sample_18hr[under_sample_18hr['hospital_expire_flag'] == 0]))


Under sampled Deceased 6hr:   4783

Under sampled Dishcharged 6hr:   4783

Under sampled Deceased 12hr:   5342

Under sampled Dishcharged 12hr:   5342

Under sampled Deceased 18hr:   5466

Under sampled Dishcharged 18hr:   5466


In [39]:
ICU_6hr_df = under_sample_6hr
ICU_6hr_df.describe()

ICU_12hr_df = under_sample_12hr
ICU_12hr_df.describe()

ICU_18hr_df = under_sample_18hr
ICU_18hr_df.describe()

,index,subject_id,hadm_id,expire_flag,deathtime_hours,age,icustay_num,heartrate_mean,sysbp_mean,diasbp_mean,...,platelet_max,potassium_max,ptt_max,inr_max,sodium_max,bun_max,wbc_max,urineoutput,endotrachflag_mean,hospital_expire_flag
count,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,...,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000,10932.000000
mean,23360.993048,35336.350165,149577.801592,0.686242,6945.424279,64.586901,1.616356,87.960049,116.411724,59.703785,...,231.997462,4.507877,46.781682,1.818013,140.157971,33.606178,14.691720,1500.992293,0.387960,0.500000
std,13740.387839,28480.581961,28908.764429,0.464040,10804.816193,15.908319,1.526127,16.931436,17.767490,11.398988,...,132.147750,0.809826,29.483034,1.487295,5.302336,26.287005,14.709008,1298.456885,0.438298,0.500023
min,3.000000,9.000000,100011.000000,0.000000,-44.000000,17.000000,1.000000,36.578947,52.145833,23.066667,...,6.000000,1.900000,16.900000,0.600000,111.000000,1.000000,0.100000,-2425.000000,0.000000,0.000000
25%,11489.500000,12712.750000,124658.500000,0.000000,124.000000,54.000000,1.000000,75.470395,103.842105,51.919167,...,147.000000,4.000000,29.600000,1.200000,137.000000,15.000000,8.800000,680.750000,0.000000,0.000000
50%,22996.500000,25553.000000,149240.500000,1.000000,824.500000,67.000000,1.000000,86.749123,114.156250,58.681391,...,211.000000,4.400000,38.700000,1.500000,140.000000,25.000000,12.600000,1300.000000,0.000000,0.500000
75%,35195.500000,57569.000000,174784.500000,1.000000,12191.372949,78.000000,2.000000,99.110586,127.421930,66.395011,...,287.000000,4.900000,45.200000,1.700000,143.000000,44.000000,17.500000,1922.250000,0.941176,1.000000
max,47890.000000,99957.000000,199998.000000,1.000000,99165.000000,88.000000,38.000000,153.037037,193.618421,129.750000,...,1667.000000,12.100000,150.000000,42.000000,180.000000,235.000000,665.600000,39420.000000,1.000000,1.000000


In [40]:
columns1=list(ICU_6hr_df)
print(columns1)

['index', u'subject_id', u'hadm_id', u'dbsource', u'expire_flag', u'deathtime_hours', u'age', u'ethnicity', u'admission_type', u'icustay_num', u'heartrate_mean', u'sysbp_mean', u'diasbp_mean', u'meanbp_mean', u'resprate_mean', u'tempc_mean', u'spo2_mean', u'glucose_mean', u'heartrate_min', u'sysbp_min', u'diasbp_min', u'meanbp_min', u'resprate_min', u'tempc_min', u'spo2_min', u'glucose_min', u'heartrate_max', u'sysbp_max', u'diasbp_max', u'meanbp_max', u'resprate_max', u'tempc_max', u'spo2_max', u'glucose_max', u'gcs_mean', u'gcsmotor_mean', u'gcsverbal_mean', u'gcseyes_mean', u'gcs_min', u'gcsmotor_min', u'gcsverbal_min', u'gcseyes_min', u'endotrachflag_min', u'gcs_max', u'gcsmotor_max', u'gcsverbal_max', u'gcseyes_max', u'endotrachflag_max', u'po2_mean', u'pco2_mean', u'ph_mean', u'totalco2_mean', u'aniongap_mean', u'bicarbonate_mean', u'creatinine_mean', u'chloride_mean', u'hematocrit_mean', u'hemoglobin_mean', u'platelet_mean', u'potassium_mean', u'ptt_mean', u'inr_mean', u'sodium_

In [41]:
ICU_6hr_df.rename(index=str, columns={'sodium_max.1': 'sodium_max_labs', 'sodium_min.1': 'sodium_min_labs','potassium_min.1': 'potassium_min_labs', 'potassium_max.1': 'potassium_max_labs', 'lactate_min.1':'lactate_min_labs', 'lactate_max.1':'lactate_max_labs', 'hemoglobin_min.1':'hemoglobin_min_labs', 'hemoglobin_max.1':'hemoglobin_max_labs',  'hematocrit_min.1':'hematocrit_min_labs', 'hematocrit_max.1':'hematocrit_max_labs', 'chloride_min.1':'chloride_min_labs',  'chloride_max.1':'chloride_max_labs',  'bicarbonate_min.1':'bicarbonate_min_labs', 'bicarbonate_max.1':'bicarbonate_max_labs', 'bicarbonate_min.1':'bicarbonate_min_labs'}, inplace = True)
ICU_12hr_df.rename(index=str, columns={'sodium_max.1': 'sodium_max_labs', 'sodium_min.1': 'sodium_min_labs','potassium_min.1': 'potassium_min_labs', 'potassium_max.1': 'potassium_max_labs', 'lactate_min.1':'lactate_min_labs', 'lactate_max.1':'lactate_max_labs', 'hemoglobin_min.1':'hemoglobin_min_labs', 'hemoglobin_max.1':'hemoglobin_max_labs',  'hematocrit_min.1':'hematocrit_min_labs', 'hematocrit_max.1':'hematocrit_max_labs', 'chloride_min.1':'chloride_min_labs',  'chloride_max.1':'chloride_max_labs',  'bicarbonate_min.1':'bicarbonate_min_labs', 'bicarbonate_max.1':'bicarbonate_max_labs', 'bicarbonate_min.1':'bicarbonate_min_labs'}, inplace = True)
ICU_18hr_df.rename(index=str, columns={'sodium_max.1': 'sodium_max_labs', 'sodium_min.1': 'sodium_min_labs','potassium_min.1': 'potassium_min_labs', 'potassium_max.1': 'potassium_max_labs', 'lactate_min.1':'lactate_min_labs', 'lactate_max.1':'lactate_max_labs', 'hemoglobin_min.1':'hemoglobin_min_labs', 'hemoglobin_max.1':'hemoglobin_max_labs',  'hematocrit_min.1':'hematocrit_min_labs', 'hematocrit_max.1':'hematocrit_max_labs', 'chloride_min.1':'chloride_min_labs',  'chloride_max.1':'chloride_max_labs',  'bicarbonate_min.1':'bicarbonate_min_labs', 'bicarbonate_max.1':'bicarbonate_max_labs', 'bicarbonate_min.1':'bicarbonate_min_labs'}, inplace = True)

In [42]:
ICU_6hr_df.head()

,index,subject_id,hadm_id,dbsource,expire_flag,deathtime_hours,age,ethnicity,admission_type,icustay_num,...,potassium_max,ptt_max,inr_max,sodium_max,bun_max,wbc_max,urineoutput,gender,endotrachflag_mean,hospital_expire_flag
3,3,9,150750,carevue,1,107.0,41,UNKNOWN/NOT SPECIFIED,EMERGENCY,1,...,2.9,40.001833,1.582101,138.0,27.366373,12.974447,1450.0,M,1.0,1
5,5,12,112213,carevue,1,283.0,72,WHITE,ELECTIVE,1,...,4.6,47.500000,1.600000,145.0,28.000000,8.400000,345.0,M,1.0,1
17,17,31,128652,carevue,1,157.0,72,WHITE,EMERGENCY,1,...,3.7,28.500000,1.100000,133.0,11.000000,12.974447,2300.0,M,1.0,1
48,48,84,166401,carevue,1,0.0,44,WHITE,EMERGENCY,1,...,3.9,40.001833,1.582101,138.0,7.000000,11.600000,2395.0,F,1.0,1
53,53,91,121205,carevue,1,309.0,81,WHITE,EMERGENCY,1,...,5.1,42.900000,2.400000,141.0,47.000000,7.300000,305.0,F,0.0,1


In [43]:
ICU_6hr_df.drop(columns=['subject_id'], inplace=True)
ICU_12hr_df.drop(columns=['subject_id'], inplace=True)
ICU_18hr_df.drop(columns=['subject_id'], inplace=True)

In [44]:
ICU_6hr_df['respxvent'] = ICU_6hr_df['endotrachflag_mean'] * ICU_6hr_df['resprate_mean']
ICU_12hr_df['respxvent'] = ICU_12hr_df['endotrachflag_mean'] * ICU_12hr_df['resprate_mean']
ICU_18hr_df['respxvent'] = ICU_18hr_df['endotrachflag_mean'] * ICU_18hr_df['resprate_mean']

In [45]:
scaler = preprocessing.MinMaxScaler()

In [46]:
scaled_6hr_df = scaler.fit_transform(ICU_6hr_df.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type' ], axis=1))
scaled_12hr_df = scaler.fit_transform(ICU_12hr_df.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type' ], axis=1))
scaled_18hr_df = scaler.fit_transform(ICU_18hr_df.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type' ], axis=1))

/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [47]:
scaled_6hr_df = pd.DataFrame(scaled_6hr_df, columns=list(ICU_6hr_df.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type' ], axis=1)))
scaled_12hr_df = pd.DataFrame(scaled_12hr_df, columns=list(ICU_12hr_df.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type' ], axis=1)))
scaled_18hr_df = pd.DataFrame(scaled_18hr_df, columns=list(ICU_18hr_df.drop(['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type' ], axis=1)))

In [48]:
df_6hr_cat = ICU_6hr_df[['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type']].copy().reset_index()
df_12hr_cat = ICU_12hr_df[['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type']].copy().reset_index()
df_18hr_cat = ICU_18hr_df[['gender', 'endotrachflag_mean', 'hospital_expire_flag','dbsource','ethnicity','admission_type']].copy().reset_index()

In [49]:
df_6hr_cat.head()

,index,gender,endotrachflag_mean,hospital_expire_flag,dbsource,ethnicity,admission_type
0,3,M,1.0,1,carevue,UNKNOWN/NOT SPECIFIED,EMERGENCY
1,5,M,1.0,1,carevue,WHITE,ELECTIVE
2,17,M,1.0,1,carevue,WHITE,EMERGENCY
3,48,F,1.0,1,carevue,WHITE,EMERGENCY
4,53,F,0.0,1,carevue,WHITE,EMERGENCY


In [50]:
scaled_6hr_df = scaled_6hr_df.reset_index()
scaled_12hr_df = scaled_12hr_df.reset_index()
scaled_18hr_df = scaled_18hr_df.reset_index()


In [51]:
scaled_6hr_df.head()

,level_0,index,hadm_id,expire_flag,deathtime_hours,age,icustay_num,heartrate_mean,sysbp_mean,diasbp_mean,...,hemoglobin_max,platelet_max,potassium_max,ptt_max,inr_max,sodium_max,bun_max,wbc_max,urineoutput,respxvent
0,0,0.000077,0.507505,1.0,0.001369,0.338028,0.0,0.399603,0.660352,0.521638,...,0.664671,0.135389,0.090090,0.170451,0.021358,0.430769,0.099496,0.019346,0.123867,0.282785
1,1,0.000129,0.122124,1.0,0.003252,0.774648,0.0,0.441327,0.413554,0.360098,...,0.622754,0.049338,0.243243,0.226998,0.021792,0.538462,0.101887,0.012472,0.057100,0.314258
2,2,0.000439,0.286519,1.0,0.001904,0.774648,0.0,0.122114,0.415482,0.347125,...,0.485684,0.107702,0.162162,0.083710,0.009685,0.353846,0.037736,0.019346,0.175227,0.240343
3,3,0.001239,0.664020,1.0,0.000225,0.380282,0.0,0.512266,0.431328,0.526479,...,0.574850,0.135981,0.180180,0.170451,0.021358,0.430769,0.022642,0.017280,0.180967,0.217454
4,4,0.001368,0.212046,1.0,0.003530,0.901408,0.0,0.538510,0.358556,0.265825,...,0.413174,0.036101,0.288288,0.192308,0.041162,0.476923,0.173585,0.010819,0.054683,0.000000


In [52]:
ICU_6hr_df = pd.concat([scaled_6hr_df, df_6hr_cat], axis=1)
ICU_12hr_df = pd.concat([scaled_12hr_df, df_12hr_cat], axis=1)
ICU_18hr_df = pd.concat([scaled_18hr_df, df_18hr_cat], axis=1)

In [53]:
ICU_6hr_df.head()

,level_0,index,hadm_id,expire_flag,deathtime_hours,age,icustay_num,heartrate_mean,sysbp_mean,diasbp_mean,...,wbc_max,urineoutput,respxvent,index,gender,endotrachflag_mean,hospital_expire_flag,dbsource,ethnicity,admission_type
0,0,0.000077,0.507505,1.0,0.001369,0.338028,0.0,0.399603,0.660352,0.521638,...,0.019346,0.123867,0.282785,3,M,1.0,1,carevue,UNKNOWN/NOT SPECIFIED,EMERGENCY
1,1,0.000129,0.122124,1.0,0.003252,0.774648,0.0,0.441327,0.413554,0.360098,...,0.012472,0.057100,0.314258,5,M,1.0,1,carevue,WHITE,ELECTIVE
2,2,0.000439,0.286519,1.0,0.001904,0.774648,0.0,0.122114,0.415482,0.347125,...,0.019346,0.175227,0.240343,17,M,1.0,1,carevue,WHITE,EMERGENCY
3,3,0.001239,0.664020,1.0,0.000225,0.380282,0.0,0.512266,0.431328,0.526479,...,0.017280,0.180967,0.217454,48,F,1.0,1,carevue,WHITE,EMERGENCY
4,4,0.001368,0.212046,1.0,0.003530,0.901408,0.0,0.538510,0.358556,0.265825,...,0.010819,0.054683,0.000000,53,F,0.0,1,carevue,WHITE,EMERGENCY


In [54]:
ICU_6hr_df['gender'] = ICU_6hr_df['gender'].replace('F',0)
ICU_6hr_df['gender'] = ICU_6hr_df['gender'].replace('M',1)

In [55]:
ICU_12hr_df['gender'] = ICU_12hr_df['gender'].replace('F',0)
ICU_12hr_df['gender'] = ICU_12hr_df['gender'].replace('M',1)

In [56]:
ICU_18hr_df['gender'] = ICU_18hr_df['gender'].replace('F',0)
ICU_18hr_df['gender'] = ICU_18hr_df['gender'].replace('M',1)

In [57]:
print(ICU_6hr_df.shape)
print(ICU_12hr_df.shape)
print(ICU_18hr_df.shape)

(9566, 105)
(10684, 111)
(10932, 111)


In [58]:
y6 = ICU_6hr_df.hospital_expire_flag
y12 = ICU_12hr_df.hospital_expire_flag
y18 = ICU_18hr_df.hospital_expire_flag

In [59]:
print(ICU_6hr_df.columns)
ICU_6hr_df.drop(['hospital_expire_flag', 'level_0', 'index','dbsource','ethnicity','admission_type' ], axis=1, inplace=True)

print(ICU_12hr_df.columns)
ICU_12hr_df.drop(['hospital_expire_flag', 'level_0', 'index','dbsource','ethnicity','admission_type' ], axis=1, inplace=True)

print(ICU_18hr_df.columns)
ICU_18hr_df.drop(['hospital_expire_flag', 'level_0', 'index','dbsource','ethnicity','admission_type' ], axis=1, inplace=True)


Index([             u'level_0',                u'index',
                    u'hadm_id',          u'expire_flag',
            u'deathtime_hours',                  u'age',
                u'icustay_num',       u'heartrate_mean',
                 u'sysbp_mean',          u'diasbp_mean',
       ...
                    u'wbc_max',          u'urineoutput',
                  u'respxvent',                u'index',
                     u'gender',   u'endotrachflag_mean',
       u'hospital_expire_flag',             u'dbsource',
                  u'ethnicity',       u'admission_type'],
      dtype='object', length=105)
Index([             u'level_0',                u'index',
                    u'hadm_id',          u'expire_flag',
            u'deathtime_hours',                  u'age',
                u'icustay_num',       u'heartrate_mean',
                 u'sysbp_mean',          u'diasbp_mean',
       ...
                    u'wbc_max',          u'urineoutput',
                  u'respxvent',

In [60]:
X6_train, X6_test, Y6_train, Y6_test = train_test_split(ICU_6hr_df, y6, test_size=0.2, random_state=43)
X12_train, X12_test, Y12_train, Y12_test = train_test_split(ICU_12hr_df, y12, test_size=0.2, random_state=43)
X18_train, X18_test, Y18_train, Y18_test = train_test_split(ICU_18hr_df, y18, test_size=0.2, random_state=43)

In [61]:
X6_train['gender'].replace('F',0,inplace=True)
X6_train['gender'].replace('M',1,inplace=True)

X6_test['gender'].replace('F',0,inplace=True)
X6_test['gender'].replace('M',1,inplace=True)

X12_train['gender'].replace('F',0,inplace=True)
X12_train['gender'].replace('M',1,inplace=True)

X12_test['gender'].replace('F',0,inplace=True)
X12_test['gender'].replace('M',1,inplace=True)


X18_train['gender'].replace('F',0,inplace=True)
X18_train['gender'].replace('M',1,inplace=True)

X18_test['gender'].replace('F',0,inplace=True)
X18_test['gender'].replace('M',1,inplace=True)

/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [62]:
print(X6_train.shape)
print(Y6_train.shape)
print(X6_test.shape)
print(Y6_test.shape)

print(X12_train.shape)
print(Y12_train.shape)
print(X12_test.shape)
print(Y12_test.shape)

print(X18_train.shape)
print(Y18_train.shape)
print(X18_test.shape)
print(Y18_test.shape)

(7652, 98)
(7652,)
(1914, 98)
(1914,)
(8547, 104)
(8547,)
(2137, 104)
(2137,)
(8745, 104)
(8745,)
(2187, 104)
(2187,)


In [63]:
X6_train.to_csv('./Data/X6_train.csv', index=False)
Y6_train.to_csv('./Data/Y6_train.csv', header=['Y'])
X6_test.to_csv('./Data/X6_test.csv', index=False)
Y6_test.to_csv('./Data/Y6_test.csv', header=['Y'])

X12_train.to_csv('./Data/X12_train.csv', index=False)
Y12_train.to_csv('./Data/Y12_train.csv', header=['Y'])
X12_test.to_csv('./Data/X12_test.csv', index=False)
Y12_test.to_csv('./Data/Y12_test.csv', header=['Y'])

X18_train.to_csv('./Data/X18_train.csv', index=False)
Y18_train.to_csv('./Data/Y18_train.csv', header=['Y'])
X18_test.to_csv('./Data/X18_test.csv', index=False)
Y18_test.to_csv('./Data/Y18_test.csv', header=['Y'])

In [64]:
print("expired 6hr: " + str(Y6_train.sum()))
print("Alive 6hr: " + str(Y6_train.size - Y6_train.sum() ))
print("\n")
print("expired 12hr: " + str(Y12_train.sum()))
print("Alive 12hr: " + str(Y12_train.size - Y12_train.sum() ))
print("\n")
print("expired 18hr: " + str(Y18_train.sum()))
print("Alive 18hr: " + str(Y18_train.size - Y18_train.sum() ))

expired 6hr: 3817
Alive 6hr: 3835


expired 12hr: 4285
Alive 12hr: 4262


expired 18hr: 4336
Alive 18hr: 4409
